### Analyses code

### Sanittawan Tan

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# import warnings
# warnings.filterwarnings('ignore')

In [4]:
prelim_dtype = {'year': 'Int64', 'country': 'string', 'Status': 'string'}
dem = pd.read_csv('./democracy.csv')

In [5]:
dem.head()

,country,year,PR,CL,Status,FH_category,FH_F,FH_NF,FH_PF,BMR_democracy,...,v2elpubfin_ord,v2elembaut_ord,v2elmulpar_ord,v2elvotbuy_ord,v2elfrcamp_ord,v2elfrfair_ord,v2elaccept_ord,v2elasmoff_ord,v2elintim,is_erosion
0,afghanistan,2008,5,6,NF,0,0,1,0,0.0,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,afghanistan,2009,6,6,NF,0,0,1,0,0.0,...,0.0,2.0,3.0,0.0,2.0,2.0,3.0,2.0,-0.421,0
2,afghanistan,2010,6,6,NF,0,0,1,0,0.0,...,0.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,-0.376,0
3,afghanistan,2011,6,6,NF,0,0,1,0,0.0,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,afghanistan,2012,6,6,NF,0,0,1,0,0.0,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [8]:
nominal = ['v2exl_legitideolcr_0', 'v2exl_legitideolcr_1', 
            'v2exl_legitideolcr_2', 'v2exl_legitideolcr_3',
            'v2exl_legitideolcr_4', 'v2lgfunds', 'v2lgamend',
            'v2exhoshog', 'v2exaphogp', 'v2exaphos', 'v2ddlexci',
            'v2ddlexrf', 'v2ddlexpl'
]

In [12]:
dem['v2exl_legitideolcr_2']

0       0.000
1       0.000
2       0.000
3       0.000
4       0.000
5       0.000
6       0.000
7       0.000
8       0.000
9       0.000
10      0.429
11      0.429
12      0.429
13      0.429
14      0.429
15      0.286
16      0.286
17      0.286
18      0.286
19      0.286
20      0.250
21      0.250
22      0.250
23      0.250
24      0.250
25      0.250
26      0.250
27      0.250
28      0.250
29      0.250
        ...  
1916    0.600
1917    0.600
1918    0.600
1919    0.400
1920    0.600
1921    0.600
1922    0.400
1923    0.400
1924    0.200
1925    0.200
1926    0.500
1927    0.500
1928    0.500
1929    0.500
1930    0.500
1931    0.500
1932    0.500
1933    0.400
1934    0.400
1935    0.400
1936    0.000
1937    0.000
1938    0.000
1939    0.000
1940    0.000
1941    0.000
1942    0.000
1943    0.000
1944    0.000
1945    0.000
Name: v2exl_legitideolcr_2, Length: 1946, dtype: float64